In [12]:
import sys
from pprint import pprint

import pybmd

from python_get_resolve import GetResolve

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
sub_folder = media_pool.GetRootFolder().GetSubFolderList()[0]

# create timeline 1
project.SetSetting('timelineResolutionWidth', "1920")
project.SetSetting('timelineResolutionHeight', "1080")
# project.SetSetting('timelineFrameRate', 25)
media_pool.CreateEmptyTimeline("timeline 1")
while True:
    terminate_input = input("请取消勾选 timeline 1 的 use project setting，取消勾选完毕请输入 OK。程序将继续运行。")
    if terminate_input == "OK":
        break
media_pool.AppendToTimeline(sub_folder.GetClipList()[0])
project.GetCurrentTimeline().SetSetting("timelineResolutionWidth", "3840")
project.GetCurrentTimeline().SetSetting("timelineResolutionHeight", "2160")
if project.SetSetting('timelineFrameRate', float(25)):
    print("set timeline frame rate successful")
print(f"timeline 1 width: {project.GetCurrentTimeline().GetSetting()['timelineResolutionWidth']}")
print(f"timeline 1 height: {project.GetCurrentTimeline().GetSetting()['timelineResolutionHeight']}")
print(f"timeline 1 frame rate: {project.GetCurrentTimeline().GetSetting()['timelineFrameRate']}")
#
# # create timeline 2
# project.SetSetting('timelineResolutionWidth', "4096")
# project.SetSetting('timelineResolutionHeight', "2160")
# # media_pool.CreateTimelineFromClips("timeline 2", sub_folder.GetClipList())
# media_pool.CreateEmptyTimeline("timeline 2")
# media_pool.AppendToTimeline(sub_folder.GetClipList()[1])
# print(f"timeline 2 width: {project.GetCurrentTimeline().GetSetting()['timelineResolutionWidth']}")
# print(f"timeline 2 height: {project.GetCurrentTimeline().GetSetting()['timelineResolutionHeight']}")

timeline 1 width: 3840
timeline 1 height: 2160
timeline 1 frame rate: 24


In [7]:
while True:
    x = input("what do you want?")
    if x == "Y":
        print("ok!")
        break

ok!


In [11]:

from pybmd import Bmd

resolve = Bmd()
project = resolve.get_project_manager().get_current_project()
x = project.get_setting()['timelineFrameRate']
print(type(x))

<class 'float'>
